In [2]:
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import os

d:\Anaconda\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
d:\Anaconda\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [3]:

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

In [5]:
import sounddevice as sd
from scipy.io.wavfile import write
import numpy as np

# Set recording parameters
duration = 5  # seconds
sample_rate = 16000  # Wav2Vec2 model requires 16kHz audio

# Record audio from the microphone
print("Recording... Please speak.")
audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
sd.wait()  # Wait until recording is finished
print("Recording complete.")

# Convert audio to the format expected by Wav2Vec2
waveform = torch.tensor(audio_data).squeeze()

# Process and tokenize the waveform
inputs = processor(waveform, sampling_rate=sample_rate, return_tensors="pt", padding=True)
input_values = inputs.input_values

# Get the logits from the model
with torch.no_grad():
    logits = model(input_values).logits

# Decode predicted ids to text
predicted_ids = torch.argmax(logits, dim=-1)
predicted_text = processor.batch_decode(predicted_ids)[0]
print("Predicted Text:", predicted_text)


Recording... Please speak.
Recording complete.
Predicted Text: HALLO DEFIUS MONOGH FROM PEROAGO


In [6]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_pronunciation_with_confidence(ground_truth, prediction, logits):
    ground_truth_words = ground_truth.split()
    predicted_words = prediction.split()
    
    # Calculate confidence scores (softmax on logits for each word)
    softmax = torch.nn.Softmax(dim=-1)
    confidence_scores = softmax(logits).max(dim=-1).values.squeeze().tolist()
    
    # Match confidence score to each predicted word
    fig, ax = plt.subplots()
    for idx, word in enumerate(ground_truth_words):
        if idx < len(predicted_words):
            # Green for correct, Red for incorrect, Gradient based on confidence
            correct = word.lower() == predicted_words[idx].lower()
            confidence = confidence_scores[idx] if idx < len(confidence_scores) else 1.0
            color = (1 - confidence, confidence, 0) if correct else (confidence, 0, 0)
            ax.text(idx * 0.5, 0.5, word, color=color, fontsize=12, ha='center')
        else:
            ax.text(idx * 0.5, 0.5, word, color="gray", fontsize=12, ha='center')
    ax.axis('off')
    plt.show()

# Call the function with confidence-based coloring
visualize_pronunciation_with_confidence(ground_truth_text, predicted_text, logits)


NameError: name 'ground_truth_text' is not defined